In [ ]:
using Newtonsoft.Json;

var input = System.IO.File.ReadAllLines("input.txt");

input

index,value
0,be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
1,edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
2,fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
3,fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
4,aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
5,fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
6,dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
7,bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
8,egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
9,gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce


In [ ]:
public class ScrambledDigits{
    public List<string> InputValues {get;set;}
    public List<string> OutputValues {get;set;}

    public Dictionary<int,string> DecodedDigits {get;set;}

    public ScrambledDigits(string inputString){
        var set = inputString.Split("|");
        InputValues = set[0].Trim().Split(" ").Select(v => ReorderValue(v)).ToList();
        OutputValues = set[1].Trim().Split(" ").Select(v => ReorderValue(v)).ToList();

        // Console.WriteLine(JsonConvert.SerializeObject(InputValues));
        // Console.WriteLine(JsonConvert.SerializeObject(OutputValues));
    }

    string ReorderValue(string v) => new string(v.OrderBy(c => c).ToArray());

    public List<int> DecodedOutput(){
        var list = new List<int>();

        foreach(var o in OutputValues){
            if(DecodedDigits.ContainsValue(o)){
                list.Add(DecodedDigits.First(x => x.Value == o).Key);
            }
        }

        return list;
    }
}

public string Add(string a, string b){
    var ac = a.ToList();
    var bc = b.ToList();
    return new string(ac.Union(bc).Distinct().OrderBy(x => x).ToArray());
}

public string Subtract(string a, string b){
    var ac = a.ToList();
    var bc = b.ToList();
    
    return new string(ac.Except(bc).OrderBy(x => x).ToArray());
}


In [ ]:
public void DecodeAllDigits(ScrambledDigits s){
    var DecodedDigits = new Dictionary<int,string>();
    
    //Solve 1, 4, 7, 8
    var uniqueDigits = s.InputValues
        .Where(x => new[]{2,3,4,7}.Contains(x.Length));
    foreach(var v in uniqueDigits){
        // Console.WriteLine(v);
        if(v.Length == 2){
            DecodedDigits.Add(1,v);
        }else if(v.Length == 3){
            DecodedDigits.Add(7,v);
        }else if(v.Length == 4){
            DecodedDigits.Add(4,v);
        }else if(v.Length == 7){
            DecodedDigits.Add(8,v);
        }
    }
    
    var remainingDigits = s.InputValues.Except(uniqueDigits).ToList();
    
    //Solve 9
    var abcdf = Add(DecodedDigits[7],DecodedDigits[4]);
    DecodedDigits.Add(9,remainingDigits.First(x => x.Intersect(abcdf).Count() == 5));
    remainingDigits.Remove(DecodedDigits[9]);
    
    //Solve 6
    var bdeg = Subtract(DecodedDigits[8],DecodedDigits[7]);
    DecodedDigits.Add(6, remainingDigits.First(x 
            => x.Length == 6
            && x.Intersect(bdeg).Count() == 4));
    remainingDigits.Remove(DecodedDigits[6]);
    
    //Solve 0
    DecodedDigits.Add(0,remainingDigits.First(x => x.Length == 6));
    remainingDigits.Remove(DecodedDigits[0]);
    
    //Solve 3
    var d = Subtract(DecodedDigits[8],DecodedDigits[0]);
    var acdf = Add(DecodedDigits[7],d);
    DecodedDigits.Add(3,remainingDigits.First(x => x.Intersect(acdf).Count() == 4));
    remainingDigits.Remove(DecodedDigits[3]);
    
    //Solve 5
    DecodedDigits.Add(5,remainingDigits.First(x => x.Intersect(DecodedDigits[6]).Count() == 5));
    remainingDigits.Remove(DecodedDigits[5]);
    
    //Solve 2
    DecodedDigits.Add(2,remainingDigits.First());
    
    s.DecodedDigits = DecodedDigits;
}


In [ ]:
var digitSets = input.Select(x => {
    var digits = new ScrambledDigits(x);
    DecodeAllDigits(digits);
    return digits;
}).ToList();

(digitSets.Select(x => long.Parse(string.Join("",x.DecodedOutput()))).Sum())


61229